In [193]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import re
import spacy
import json
from sklearn.metrics import classification_report
import pandas as pd
from pathlib import Path
from collections import defaultdict, Counter


In [194]:
#Pipeline Configuration

#Selected usage of single components
use_spacy = True
use_regex = False
use_piiranha = False

# Select the spaCy model

# Load the spaCy model depending on your setup
# Uncomment one of the following if needed

# Trained on synthetic data (SynthB)
#nlp = spacy.load("../custom_spacy_model_synthetic_data_b_push")
#nlp_name = "synthetic_b"

# Trained on original data (OrigData)
nlp = spacy.load("../custom_spacy_model_doccano_labeling")
nlp_name = "orginal"

# Untrained base model
#nlp = spacy.load("de_core_news_md")
#nlp_name = "untrained"

# Toggle for activating EntityRuler
use_ruler = True


Label Mapping der einzelnen Identifier

In [195]:
# Define priority for conflict resolution: higher values indicate stronger precedence
PRIORITY_MAP = {
    "regex": 3,       # Regex detections have the highest priority
    "piiranha": 1,    # Piiranha has the lowest priority
    "spacy": 2        # spaCy takes middle priority
}

# Define the set of labels that should be extracted
TARGET_LABELS = [
    "TITEL", "VORNAME", "NACHNAME", "FIRMA", "TELEFONNUMMER", "EMAIL", "FAX", "STRASSE",
    "HAUSNUMMER", "POSTLEITZAHL", "WOHNORT", "ZÄHLERNUMMER", "ZÄHLERSTAND", "VERTRAGSNUMMER",
    "ZAHLUNG", "BANK", "IBAN", "BIC", "DATUM", "GESENDET_MIT", "LINK"
]

# Maps labels from different sources (spaCy, Piiranha) to unified categories
LABEL_MAP = {
    # spaCy labels
    "PER": "NAME",
    "LOC": "ADRESSE",
    "ORG": "FIRMA",
    "DATE": "DATUM",
    "TIME": "DATUM",
    "GPE": "ADRESSE",
    "NORP": "GRUPPE",
    "MONEY": "ZAHLUNG",

    # Piiranha labels
    "I-GIVENNAME": "NAME",
    "I-SURNAME": "NAME",
    "I-DATEOFBIRTH": "DATUM",
    "I-EMAIL": "KONTAKT",
    "I-TELEPHONENUM": "KONTAKT",
    "I-USERNAME": "KONTAKT",
    "I-CREDITCARDNUMBER": "ZAHLUNG",
    "I-ACCOUNTNUM": "VERTRAG",
    "I-BILLINGNUM": "VERTRAG",
    "I-IDCARDNUM": "VERTRAG",
    "I-TAXNUM": "VERTRAG",
    "I-CITY": "ADRESSE",
    "I-ZIPCODE": "ADRESSE",
    "I-STREET": "ADRESSE",
    "I-BUILDINGNUM": "ADRESSE"
}

# Regular expression patterns used for matching key entity types
REGEX_PATTERNS = {
    # Strong and well-defined email pattern
    "EMAIL": r"\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z]{2,}\b",

    # IBAN pattern specific to German format (22 characters)
    "IBAN": r"\bDE\d{20}\b",

    # BIC (SWIFT code) format
    "BIC": r"\b[A-Z]{6}[A-Z2-9][A-NP-Z0-9]{1}([A-Z0-9]{3})?\b",

    # URL pattern: includes http, https, or www
    "URL": r"\bhttps?://[^\s]+|www\.[^\s]+\b",

    # Contract number pattern: must be prefixed by a keyword and contain digits
    "VERTRAG": r"\b(vertragsnummer|vertragsnr\.?|vnr|vn)[\s:]{1,3}\d{7,10}\b",

    # Well-formed dates, including ISO and German styles
    "DATUM": (
        r"\b\d{2}\.\d{2}\.\d{4}\b|"      # e.g., 15.08.2024
        r"\b\d{4}-\d{2}-\d{2}\b|"        # e.g., 2024-08-15
        r"\b(19|20)\d{2}\b"              # Four-digit years
    ),

    # German phone number format, must include country code
    "TELEFON": r"\b\+49\s?\d[\d\s/-]{6,}\b",

    # Alphanumeric meter numbers, typically longer than 10 characters
    "ZÄHLERNUMMER": r"\b[A-Z]{2}[A-Z0-9]{8,}\b",

    # Payment amount pattern: includes currency keywords
    "ZAHLUNG": r"\b\d{1,5}[.,]\d{2}\s?(€|Euro|EUR|Cent)\b",

    # Street pattern: must end with common street suffixes
    "STRASSE": r"\b\w+(straße|gasse|allee|weg|platz|str\.|grund)\b"
}

PIIranha Spans

In [196]:
# Load the Piiranha model and tokenizer from HuggingFace
model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Select device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to extract entity spans from text using Piiranha
def get_piiranha_spans(text):
    # Tokenize input and get offset mappings (to map tokens back to character positions)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_offsets_mapping=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    offset_mapping = inputs.pop("offset_mapping")[0].tolist()

    # Inference: get model predictions without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)[0].tolist()

    spans = []
    current_label = None
    current_start = None

    # Iterate over each token and its offset
    for i, (start, end) in enumerate(offset_mapping):
        if start == end:
            continue  # Skip special tokens like [CLS], [SEP]

        # Get predicted label and map it to a simplified label name
        raw_label = model.config.id2label[predictions[i]]
        mapped_label = LABEL_MAP.get(raw_label, None)

        # If the label is one of the target labels, track span start and continuation
        if mapped_label in TARGET_LABELS:
            if current_label == mapped_label:
                continue  # Continue current span
            else:
                # If label changes, close previous span and start new one
                if current_label is not None:
                    spans.append({"start": current_start, "end": offset_mapping[i-1][1], "label": current_label})
                current_label = mapped_label
                current_start = start
        else:
            # If no valid label, close previous span if one was open
            if current_label is not None:
                spans.append({"start": current_start, "end": offset_mapping[i-1][1], "label": current_label})
                current_label = None
                current_start = None

    # Finalize last span if still open
    if current_label is not None:
        spans.append({"start": current_start, "end": offset_mapping[-1][1], "label": current_label})

    return spans

SpaCy Ruler laden & Spans

In [ ]:
if use_ruler:
    # Remove existing entity_ruler if present
    if "entity_ruler" in nlp.pipe_names:
        nlp.remove_pipe("entity_ruler")

    # Add a new entity_ruler before the NER component
    ruler = nlp.add_pipe("entity_ruler", before="ner")

    # Define patterns (Regex-based, no Gazetteer)
    strasse_patterns = [
        {
            "label": "STRASSE",
            "pattern": [
                {"TEXT": {"REGEX": r".*(straße|gasse|allee|weg|platz|str.|grund)$"}},
                {"TEXT": {"REGEX": r"^\d+[a-zA-Z]?$"}}
            ]
        }
    ]

    vertragsnummer_patterns = [
        {
            "label": "VERTRAGSNUMMER",
            "pattern": [
                {"LOWER": {"IN": ["vertragsnummer", "vertragsnr.", "vnr", "vn"]}},
                {"IS_PUNCT": True, "OP": "*"},
                {"TEXT": {"REGEX": r"^\d{6,12}\\.?$"}}
            ]
        }
    ]

    kundennummer_patterns = [
        {
            "label": "KUNDENNUMMER",
            "pattern": [
                {"LOWER": {"IN": ["kundennummer", "kundennr.", "kdnr", "kd"]}},
                {"IS_PUNCT": True, "OP": "*"},
                {"TEXT": {"REGEX": r"^\d{6,12}\\.?$"}}
            ]
        }
    ]

    zahlung_pattern = [
        {
            "label": "ZAHLUNG",
            "pattern": [
                {"TEXT": {"REGEX": r"^\d+[.,]?\d{0,2}$"}},
                {"TEXT": {"REGEX": r"^(\u20ac|euro|eur)$"}}
            ]
        }
    ]

    email_pattern = [
        {
            "label": "EMAIL",
            "pattern": [
                {"TEXT": {"REGEX": r"^[\w\.-]+@[\w\.-]+\.\w{2,}$"}}
            ]
        }
    ]

    telefon_pattern = [
        {
            "label": "TELEFON",
            "pattern": [
                {"TEXT": {"REGEX": r"^(\+49|0)[\d\s/-]{7,}$"}}
            ]
        }
    ]

    url_pattern = [
        {
            "label": "LINK",
            "pattern": [
                {"TEXT": {"REGEX": r"^https?://[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
            ]
        },
        {
            "label": "LINK",
            "pattern": [
                {"TEXT": {"REGEX": r"^www\.[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
            ]
        }
    ]

    datum_pattern = [
        {
            "label": "DATUM",
            "pattern": [
                {"TEXT": {"REGEX": r"^(\d{1,2}[./-]){2}\d{2,4}$"}}
            ]
        },
        {
            "label": "DATUM",
            "pattern": [
                {"TEXT": {"REGEX": r"^\d{4}-\d{2}-\d{2}$"}}
            ]
        }
    ]

    # Add all patterns to the ruler
    ruler.add_patterns(
        zahlung_pattern + url_pattern + email_pattern + telefon_pattern +
        strasse_patterns + vertragsnummer_patterns + kundennummer_patterns + datum_pattern
    )

Model with EntityRuler saved at: C:\Users\morit\OneDrive\Uni\02_Master\05_Studium\02_Semester_II\Data Analytics in Applications\VSCode\daia-eon\notebooks\3_model_training_and_testing\spacy_pipeline\piiranha_refinement\custom_spacy_model_with_ruler


In [198]:
def get_spacy_spans(text):
    doc = nlp(text)
    spans = []
    for ent in doc.ents:
        label = LABEL_MAP.get(ent.label_, ent.label_)
        if label in TARGET_LABELS:
            spans.append({"start": ent.start_char, "end": ent.end_char, "label": label})
    return spans

Regex Spans

In [199]:
def get_regex_spans(text):
    spans = []
    for raw_label, pattern in REGEX_PATTERNS.items():
        mapped_label = LABEL_MAP.get(raw_label, raw_label)  # bleibt bei IBAN = IBAN
        if mapped_label not in TARGET_LABELS:
            continue
        for match in re.finditer(pattern, text):
            spans.append({
                "start": match.start(),
                "end": match.end(),
                "label": mapped_label
            })
    return spans

In [200]:
# Beispieltext zum Testen
sample_text = """
Sehr geehrter Herr John Doe,
Ihre Kundennummer 4012345678 ist aktiv.
Bitte kontaktieren Sie uns unter max@eon.de oder +49 171 1234567.
Ihre Zahlung über 89,99 € wurde am 15-08-2024 verbucht.
"""

# PIIranha-Spans abrufen
piiranha_spans = get_piiranha_spans(sample_text)
spacy_spans = get_spacy_spans(sample_text)
regex_spans = get_regex_spans(sample_text)

# Ergebnisse ausgeben
print(piiranha_spans)
print(spacy_spans)
print(regex_spans)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
[{'start': 20, 'end': 24, 'label': 'VORNAME'}, {'start': 25, 'end': 28, 'label': 'NACHNAME'}, {'start': 48, 'end': 58, 'label': 'VERTRAGSNUMMER'}, {'start': 103, 'end': 113, 'label': 'EMAIL'}, {'start': 119, 'end': 135, 'label': 'TELEFONNUMMER'}, {'start': 154, 'end': 161, 'label': 'ZAHLUNG'}, {'start': 171, 'end': 173, 'label': 'ZAHLUNG'}]
[{'start': 103, 'end': 113, 'label': 'EMAIL'}, {'start': 177, 'end': 181, 'label': 'DATUM'}]


In [201]:
# 🧠 Optional: Merge overlapping or duplicate spans by sorting them
def merge_spans(spans):
    return sorted(spans, key=lambda x: x['start'])

# 🔄 Resolve conflicts between overlapping spans based on priority
def resolve_conflicts(spans):
    # Sort spans: first by start index, then by descending length (longer first), then by priority
    spans = sorted(
        spans,
        key=lambda x: (
            x["start"],
            -(x["end"] - x["start"]),
            -PRIORITY_MAP.get(x.get("source", ""), 0)
        )
    )

    resolved = []
    occupied = set()

    for span in spans:
        span_range = set(range(span["start"], span["end"]))
        conflict = False

        for existing in resolved:
            existing_range = set(range(existing["start"], existing["end"]))

            # ❌ If spans overlap
            if span_range & existing_range:
                # ➕ If one span is fully inside the other → decide based on priority
                if span["start"] >= existing["start"] and span["end"] <= existing["end"]:
                    if PRIORITY_MAP[span["source"]] > PRIORITY_MAP[existing["source"]]:
                        resolved.remove(existing)
                        break
                    else:
                        conflict = True
                        break
                elif existing["start"] >= span["start"] and existing["end"] <= span["end"]:
                    if PRIORITY_MAP[span["source"]] < PRIORITY_MAP[existing["source"]]:
                        conflict = True
                        break
                    else:
                        resolved.remove(existing)
                        break
                else:
                    # True overlap, not nested – reject current span
                    conflict = True
                    break

        if not conflict:
            resolved.append(span)
            occupied.update(span_range)

    return resolved

# 🔐 Apply final redaction using label numbering (e.g., NAME_1, NAME_2, ...)
def apply_final_redaction(text, spans):
    spans = sorted(spans, key=lambda x: x["start"])
    redacted = []
    last_index = 0
    label_counter = defaultdict(int)

    for span in spans:
        label = span["label"]
        label_counter[label] += 1
        label_with_id = f"{label}_{label_counter[label]}"

        # Keep text before the span
        redacted.append(text[last_index:span["start"]])
        # Insert replacement token
        redacted.append(f"[{label_with_id}]")
        # Update position pointer
        last_index = span["end"]

    # Append any remaining text
    redacted.append(text[last_index:])
    return ''.join(redacted)

# 🧩 Main masking function using multiple components (e.g., regex, spacy, piiranha)
def mask_text_with_all(text, components=["regex"]):
    all_spans = []

    if "regex" in components:
        for span in get_regex_spans(text):
            span["source"] = "regex"
            all_spans.append(span)

    if "piiranha" in components:
        for span in get_piiranha_spans(text):
            span["source"] = "piiranha"
            all_spans.append(span)

    if "spacy" in components:
        for span in get_spacy_spans(text):
            span["source"] = "spacy"
            all_spans.append(span)

    # 🔧 Resolve span conflicts and apply redaction
    spans = resolve_conflicts(all_spans)
    merged = merge_spans(spans)
    return apply_final_redaction(text, merged)

# 🎯 Mask text using only a single component (for testing or analysis)
def mask_text_with_single_component(text, component="regex"):
    if component == "regex":
        all_spans = get_regex_spans(text)
    elif component == "piiranha":
        all_spans = get_piiranha_spans(text)
    elif component == "spacy":
        all_spans = get_spacy_spans(text)
    else:
        raise ValueError(f"Unknown component: {component}")

    # Optionally resolve internal conflicts within single component spans
    spans = resolve_conflicts(all_spans)
    merged = merge_spans(spans)

    # Return the redacted version of the input text
    return apply_final_redaction(text, merged)


In [202]:
# Load ground truth test data
with open("../../../../data/original/ground_truth_split/test_norm.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Helper function: convert span dictionary to tuple
def to_tuple(span):
    return (span["start"], span["end"], span["label"])

# Evaluate predicted spans against gold standard
def evaluate_entities(pred_fn, data, threshold=0.8):
    stats = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})
    total_tp = total_fp = total_fn = 0

    for entry in data:
        text = entry["text"]
        gold_spans = [to_tuple(s) for s in entry["labels"]]
        pred_spans = [to_tuple(s) for s in pred_fn(text)]
        matched_gold = set()
        matched_pred = set()

        # Try to match each predicted span with a gold span using Jaccard similarity
        for pi, p in enumerate(pred_spans):
            best_match = None
            best_overlap = 0
            for gi, g in enumerate(gold_spans):
                if g[2] != p[2]:  # Only match if labels are the same
                    continue
                overlap = max(0, min(p[1], g[1]) - max(p[0], g[0]))
                union = max(p[1], g[1]) - min(p[0], g[0])
                jaccard = overlap / union if union > 0 else 0
                if jaccard >= threshold and jaccard > best_overlap:
                    best_overlap = jaccard
                    best_match = gi
            if best_match is not None:
                matched_gold.add(best_match)
                matched_pred.add(pi)
                stats[p[2]]["tp"] += 1
                total_tp += 1

        # Count false positives: predicted spans without gold match
        for i, p in enumerate(pred_spans):
            if i not in matched_pred:
                stats[p[2]]["fp"] += 1
                total_fp += 1

        # Count false negatives: gold spans without prediction
        for i, g in enumerate(gold_spans):
            if i not in matched_gold:
                stats[g[2]]["fn"] += 1
                total_fn += 1

    # Convert statistics to rows for DataFrame
    rows = []
    for label, counts in stats.items():
        tp, fp, fn = counts["tp"], counts["fp"], counts["fn"]
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        rows.append({
            "component": "Combined",
            "label": label,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "precision": precision,
            "recall": recall,
            "f1": f1
        })

    # Add overall performance row
    overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    overall_recall    = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    overall_f1        = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

    rows.append({
        "component": "Combined",
        "label": "OVERALL",
        "tp": total_tp,
        "fp": total_fp,
        "fn": total_fn,
        "precision": overall_precision,
        "recall": overall_recall,
        "f1": overall_f1
    })

    return pd.DataFrame(rows)

# Prediction function with configurable component selection
def run_combined_spans(text, use_spacy=use_spacy, use_regex=use_regex, use_piiranha=use_piiranha):
    all_spans = []

    if use_regex:
        for span in get_regex_spans(text):
            span["source"] = "regex"
            all_spans.append(span)

    if use_piiranha:
        for span in get_piiranha_spans(text):
            span["source"] = "piiranha"
            all_spans.append(span)

    if use_spacy:
        for span in get_spacy_spans(text):
            span["source"] = "spacy"
            all_spans.append(span)

    resolved = resolve_conflicts(all_spans)
    return merge_spans(resolved)


# Define the prediction function with the flags
df_eval = evaluate_entities(
    lambda text: run_combined_spans(
        text,
        use_spacy=use_spacy,
        use_regex=use_regex,
        use_piiranha=use_piiranha
    ),
    test_data,
    threshold=0.5
)

# Build filename based on settings
used_components = []
if use_spacy: used_components.append("spacy")
if use_regex: used_components.append("regex")
if use_piiranha: used_components.append("piiranha")
components_str = "_".join(used_components) if used_components else "none"

ruler_str = "ruler" if use_ruler else "no_ruler"

filename = f"evaluation_{components_str}_{ruler_str}_{nlp_name}.csv"

# Save evaluation results
df_eval.to_csv(filename, index=False)
print(f"Evaluation results saved to: {filename}")

Evaluation results saved to: evaluation_spacy_ruler_orginal.csv


In [203]:
df_eval

,component,label,tp,fp,fn,precision,recall,f1
0,Combined,GESENDET_MIT,5,1,1,0.833333,0.833333,0.833333
1,Combined,LINK,3,1,0,0.750000,1.000000,0.857143
2,Combined,VERTRAGSNUMMER,21,6,19,0.777778,0.525000,0.626866
3,Combined,ZAHLUNG,5,1,3,0.833333,0.625000,0.714286
4,Combined,VORNAME,57,0,0,1.000000,1.000000,1.000000
5,Combined,NACHNAME,57,2,4,0.966102,0.934426,0.950000
6,Combined,DATUM,26,11,4,0.702703,0.866667,0.776119
7,Combined,TITEL,8,0,0,1.000000,1.000000,1.000000
8,Combined,IBAN,0,0,4,0.000000,0.000000,0.000000
9,Combined,ZÄHLERNUMMER,2,2,4,0.500000,0.333333,0.400000
